In [1]:
# IMPORTANTE: Estou em fim de período no curso de Mestrado, com atividades pendentes. Não consegui
# dedicar tempo suficiente para esta atividade. Aqui vai ser mais uma exemplo da ideia do que a 
# solução em si

In [6]:
import os
import cv2
import pickle
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn import metrics
from statistics import median
from matplotlib.ticker import NullFormatter
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, jaccard_similarity_score, classification_report, \
        log_loss

%matplotlib inline

# todas as imagens serão redimensionadas para 128x128
SHAPE = 128, 128

In [3]:
# função para gerar o conjunto de treinamento dado um diretório
def train_set(folder):
    TRAINING_TYPES = ['Disguise', 'Genuine', 'Reference', 'Simulated']

#     leitura dos dados e geração das imagens binárias 128x128
    def get_training_data():
        data = {}
        for type in TRAINING_TYPES:
            data[type] = []
            subfolder = os.path.join(folder, type)
            for _, _, files in os.walk(subfolder):
                for file in files:
                    file_path = os.path.join(subfolder, file)
                    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    img = ~cv2.resize(img, SHAPE, interpolation=cv2.INTER_AREA)
                    data[type].append((file, img))
        return data
    
#     As features usadas no treinamento serão os três tipos de distância de shapes (baseadas em Hu-
#     Moments) definidas no OpenCV. Mais detalhes em: https://www.learnopencv.com/shape-matching-using-hu-moments-c-python/
    def feature_extraction(data):
        _, ref_list = zip(*data['Reference'])
        tmp_data = { "id": [], "dist1": [], "dist2": [], "dist3": [], "type": [] }
        for type in TRAINING_TYPES:
            if type != 'Reference':
                for item in data[type]:
                    name, img = item
                    name = name.split('.')[0]
                    y = name[0] if name[0] != 'S' else 'F'
                    id = int(name[1:])
                    tmp_data['id'].append(id)
                    for i in range(1, 4):
                        dists = [cv2.matchShapes(img, ref_img, i, 0) for ref_img in ref_list]
                        tmp_data['dist' + str(i)].append(median(dists))
                    tmp_data['type'].append(y)
        df = pd.DataFrame(tmp_data)
        df = df.set_index('id')
        df = df.sort_index()
        return df
    
    data = get_training_data()
    df = feature_extraction(data)
    X = df[['dist1', 'dist2', 'dist3']]
    y = df.type.values
    X = preprocessing.StandardScaler().fit(X).transform(X)
    return X, y

In [3]:
# Leitura dos dados para teste
def test_set(folder):
    TEST_TYPES = ['Reference', 'Questioned']
    
    def get_test_data():
        data = {}
        for type in TEST_TYPES:
            data[type] = []
            subfolder = os.path.join(folder, type)
            for _, _, files in os.walk(subfolder):
                for file in files:
                    file_path = os.path.join(subfolder, file)
                    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    img = ~cv2.resize(img, SHAPE, interpolation=cv2.INTER_AREA)
                    data[type].append((file, img))
        return data
    
    def feature_extraction(data):
        _, ref_list = zip(*data['Reference'])
        tmp_data = { "id": [], "dist1": [], "dist2": [], "dist3": [] }
        for item in data['Questioned']:
            name, img = item
            name = name.split('.')[0]
            id = int(name[1:])
            tmp_data['id'].append(id)
            for i in range(1, 4):
                dists = [cv2.matchShapes(img, ref_img, i, 0) for ref_img in ref_list]
                tmp_data['dist' + str(i)].append(median(dists))
        df = pd.DataFrame(tmp_data)
        df = df.set_index('id')
        df = df.sort_index()
        return df
    
    data = get_test_data()
    df = feature_extraction(data)
    X = df[['dist1', 'dist2', 'dist3']]
    X = preprocessing.StandardScaler().fit(X).transform(X)
    return X

In [15]:
# Classificador bobo, Regressão Logística com parâmetros default
def load_classifier():
    loaded_model = pickle.load(open('classifier.sav', 'rb'))
    return loaded_model

In [12]:
def train_classifier(classifier, X_train, y_train):
    return classifier.fit(X_train, y_train)

In [16]:
# Resposta no formato de uma String, conforme o README requer
def predict(classifier, X_test):
    return classifier.predict(X_test).sum()